In [30]:
#export
# see https://github.com/spulec/uncurl/blob/master/uncurl/api.py
import argparse
import json
import re
import shlex
from collections import OrderedDict, namedtuple

from six.moves import http_cookies as Cookie

parser = argparse.ArgumentParser()
parser.add_argument('command')
parser.add_argument('url')
parser.add_argument('-d', '--data')
parser.add_argument('-b', '--data-binary', default=None)
parser.add_argument('-X', default='')
parser.add_argument('-H', '--header', action='append', default=[])
parser.add_argument('--compressed', action='store_true')
parser.add_argument('--insecure', action='store_true')

BASE_INDENT = " " * 4

ParsedContext = namedtuple('ParsedContext', ['method', 'url', 'data', 'headers', 'cookies', 'verify'])

def parse_context(curl_command):
    method = "get"

    tokens = shlex.split(curl_command)
    parsed_args = parser.parse_args(tokens)

    post_data = parsed_args.data or parsed_args.data_binary
    if post_data:
        method = 'post'

    if parsed_args.X:
        method = parsed_args.X.lower()

    cookie_dict = OrderedDict()
    quoted_headers = OrderedDict()

    for curl_header in parsed_args.header:
        if curl_header.startswith(':'):
            occurrence = [m.start() for m in re.finditer(':', curl_header)]
            header_key, header_value = curl_header[:occurrence[1]], curl_header[occurrence[1] + 1:]
        else:
            header_key, header_value = curl_header.split(":", 1)

        if header_key.lower() == 'cookie':
            cookie = Cookie.SimpleCookie(header_value)
            for key in cookie:
                cookie_dict[key] = cookie[key].value
        else:
            quoted_headers[header_key] = header_value.strip()

    return ParsedContext(
        method=method,
        url=parsed_args.url,
        data=post_data,
        headers=quoted_headers,
        cookies=cookie_dict,
        verify=parsed_args.insecure
    )


def parse(curl_command, **kargs):
    parsed_context = parse_context(curl_command)

    data_token = ''
    if parsed_context.data:
        data_token = '{}data=\'{}\',\n'.format(BASE_INDENT, parsed_context.data)

    verify_token = ''
    if parsed_context.verify:
        verify_token = '\n{}verify=False'.format(BASE_INDENT)
        
        
        
    requests_kagrs=''
    for k,v in kargs.items():
        requests_kagrs += "{}{}={},\n".format(BASE_INDENT,k,str(v))

    formatter = {
        'method': parsed_context.method,
        'url': parsed_context.url,
        'data_token': data_token,
        'headers_token': "{}headers={}".format(BASE_INDENT, dict_to_pretty_string(parsed_context.headers)),
        'cookies_token': "{}cookies={}".format(BASE_INDENT, dict_to_pretty_string(parsed_context.cookies)),
        'security_token': verify_token,
        'requests_kagrs':requests_kagrs
    }

    return """requests.{method}("{url}",
{requests_kagrs}{data_token}{headers_token},
{cookies_token},{security_token}
)""".format(**formatter)

def dict_to_pretty_string(the_dict, indent=4):
    if not the_dict:
        return "{}"

    return ("\n" + " " * indent).join(
        json.dumps(the_dict, sort_keys=True, indent=indent, separators=(',', ': ')).splitlines())

In [31]:
print(parse("curl 'https://pypi.python.org/pypi/uncurl' -H 'Accept-Encoding: gzip,deflate,sdch'", timeout=0.1, allow_redirects=True))

requests.get("https://pypi.python.org/pypi/uncurl",
    timeout=0.1,
    allow_redirects=True,
    headers={
        "Accept-Encoding": "gzip,deflate,sdch"
    },
    cookies={},
)


In [22]:
d = {"timeout":True}
requests_kagrs=''
for k,v in d.items():
    requests_kagrs += (k+'='+str(v)+',')
requests_kagrs

'timeout=True,'

In [23]:
requests_kagrs=reduce(lambda a, c: "%s"%(a,c), '')

SyntaxError: invalid syntax (<ipython-input-23-56574c47f80d>, line 1)

In [18]:
!python notebook2script.py watcher.ipynb

Converted watcher.ipynb to nbexp_watcher.py


In [ ]:
from nbexp_main import timer, save_notebook
from time import sleep

def save_and_cvt(notebook_name):
    save_notebook()
    sleep(2)
    !python notebook2script.py {notebook_name}.ipynb
    save_notebook()
    
save_and_cvt('uncurl')

<IPython.core.display.Javascript object>

Converted uncurl.ipynb to nbexp_uncurl.py
